### SocialAL Models
# Fit general learning model to subset of data
KLS 12.19.22
Project info: https://osf.io/b48n2/

Model modified from :
Fareri, D. S., Chang, L. J., & Delgado, M. R. (2012). Effects of direct social experience on trust decisions and neural reward circuitry. Frontiers in Neuroscience, 6, 1–17. https://doi.org/10.3389/fnins.2012.00148

### Python version

In [1]:
import sys
print(sys.version)  

3.11.0 (v3.11.0:deaf509e8f, Oct 24 2022, 14:43:23) [Clang 13.0.0 (clang-1300.0.29.30)]


### Load modules

In [2]:
import numpy as np
import random
import math
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import itertools
import os
import csv

## Pull in functions

In [3]:
run common_functions.ipynb

In [4]:
run single_alpha_functions.ipynb

### New function to fit model to multiple subjects

In [5]:
def model_fit_mult_subject(data):
    pdt = pd.DataFrame(columns = ['Subject','alpha', 'beta', 'llh'])
    a = pd.unique(data['Subject'])
    #print(a)
    for sub in range(len(a)):
        print('Subject: ', a[sub])
        df = data[data['Subject']==a[sub]]
        df.dropna()
        df.reset_index(drop=True, inplace=True)
        dt = df.to_dict()
        params = model_fit(dt)
        #print('Params: ', params)
        line = {'Subject': sub, 'alpha':params[0], 'beta':params[1], 'llh':params[2]}
        pdt = pdt.append(line, ignore_index=True)   
    return(pdt)


### Load and clean data

In [6]:
files = os.listdir('../../data/modeling')
files.remove('.DS_Store')
files.remove('missing')
#print(files)

In [7]:
n_part = len(files)
print('number of participants: ', n_part)

number of participants:  63


In [8]:
def load_and_clean(file):
    path = os.path.join('../../data/modeling', file)
    dt = pd.read_csv(path)
    # recode trial type into numbers for model
    def stims(trial_type):
        if trial_type == "Trustworthy":
            return 0
        elif trial_type == "Neutral":
            return 1
        elif trial_type == "Untrustworthy":
            return 2
    dt['Stim_Sequence'] = dt['trial_type'].apply(stims)
    # rename response_key to choice
    def choices(response_key):
        if response_key == 'None':
            return 0 
        else:
            return response_key  
    dt['Choice'] = dt['response_key'].apply(choices)
    dt['Choice'] = pd.to_numeric(dt['Choice'])
    # calculte the trustee response
    def resp(trial_earnings):
        if trial_earnings >= 12:
            return 1
        else:
            return 0
    dt['Trustee_Response'] = dt['trial_earnings'].apply(resp)
    data = dt[['Stim_Sequence','Choice', 'Trustee_Response']]
    # remove neutral trials
    data = data[data['Stim_Sequence'] != 1]
    data.reset_index(drop=True, inplace=True)
    return(data)

In [9]:
# data = load_and_clean(files[0])
# data.head(5)

In [11]:
# create csv file
file = open('../../output/generalization_criterion/a1_fit2_2cond_part.csv', 'w')

# add header
headerList = ['id', 'alpha', 'beta', 'llh']
with open('../../output/generalization_criterion/a1_fit2_2cond_part.csv', 'a') as f_object: 
    dw = csv.DictWriter(f_object, delimiter = ',', fieldnames = headerList)
    dw.writeheader()

#loop thru participants
for x in range(n_part):
    print('loop:', x)
    subject = files[x][:-4]
    print(subject)
    
    # load and clean data
    print('loading data')
    data = load_and_clean(files[x])
    
    # fit model
    print('fitting model')
    a1mf = model_fit(data)
    a1 = np.array(a1mf.x).tolist()
    a1.insert(0,subject)
    a1.append(a1mf.fun)
    print(a1)
    
    # save results
    print('saving results')
    with open('../../output/generalization_criterion/a1_fit2_2cond_part.csv', 'a') as f_object: # create a file object
        writer_object = csv.writer(f_object) # pass the file object to csv writer
        writer_object.writerow(a1) #pass list as argument to writerow
        f_object.close() # close the file object

loop: 0
sub-2013
loading data
fitting model
['sub-2013', 0.002493050301369891, 20.0, 32.43983571576065]
saving results
loop: 1
sub-2007
loading data
fitting model
['sub-2007', 0.19693475967791718, 0.3220602867747511, 35.900563293478136]
saving results
loop: 2
sub-2006
loading data
fitting model
['sub-2006', 0.0, 1e-10, 41.588830833596695]
saving results
loop: 3
sub-2012
loading data
fitting model
['sub-2012', 0.7234889754296328, 0.19539542146568958, 37.55365861003604]
saving results
loop: 4
sub-2004
loading data
fitting model
['sub-2004', 0.133850087751657, 1.9643006880644458, 14.119346957119598]
saving results
loop: 5
sub-2010
loading data
fitting model
['sub-2010', 0.0, 1e-10, 38.81624211135692]
saving results
loop: 6
sub-2038
loading data
fitting model
['sub-2038', 9.542964382860554e-05, 12.045464959737984, 41.57930133511496]
saving results
loop: 7
sub-2039
loading data
fitting model
['sub-2039', 0.012586428810640964, 20.0, 11.188192780640822]
saving results
loop: 8
sub-2011
loading